In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/ML\ assignment\ 2

In [ ]:
from copy import *
from math import *
import h5py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics 




In [ ]:

# gnb function
def gnb_(Xtrain,Xtest,Ytrain,Ytest,LABEL_NUM):

    class_labels={}
    for i in range(LABEL_NUM):
        class_labels[i]=np.array([])

    for i in range(len(Xtrain)):
        class_labels[int(Ytrain[i][0])]=np.append(class_labels[int(Ytrain[i][0])], Xtrain[i])

    mean=np.array([])
    for i in range(len(class_labels)):
        class_labels[i]=class_labels[i].reshape(len(class_labels[i])//len(Xtrain[0]),len(Xtrain[0]))

        bb=len(class_labels[i])
        temp=class_labels[i].sum(axis=0)
        temp=temp/bb
        mean=np.append(mean,[temp])

    mean=mean.reshape(len(mean)//len(Xtrain[0]),len(Xtrain[0]))
    # calculating mean matrix for the whole dataset, for every class label

    DAD_std_dev=np.array([])
    for i in range(len(class_labels)):
        bb=len(class_labels[i])
        std_dev=np.array([])

        for j in range(len(Xtrain[0])):
            std_dev=np.append(std_dev,[0])

        std_dev=std_dev.reshape(len(Xtrain[0]),1)
        [r,c]=class_labels[i].shape

        for k in range(r):
            for l in range(c):
                std_dev[l][0]+=((class_labels[i])[k,l]-mean[i,l])**2

        std_dev=std_dev.reshape(1,len(Xtrain[0]))
        std_dev=std_dev/(bb-1)
        DAD_std_dev=np.append(DAD_std_dev,std_dev)

    DAD_std_dev=DAD_std_dev.reshape(len(DAD_std_dev)//len(Xtrain[0]),len(Xtrain[0]))
    # calculating std matrix matrix for the whole dataset, for every class label
    

    labels_prob=[0]*LABEL_NUM
    for i in range(len(Ytrain)):
        labels_prob[int(Ytrain[i][0])]+=1

    for i in range(len(labels_prob)):
        labels_prob[i]=labels_prob[i]/len(Ytrain)
    # class label probabilities


    Y_pred=np.array([])
    for i in range(len(Xtest)):
        probs_f=np.array([])

        for j in range(len(Xtrain[0])*LABEL_NUM):
            probs_f=np.append(probs_f,[0])

        probs_f=probs_f.reshape(LABEL_NUM,len(Xtrain[0]))

        # gnb function
        for j in range(LABEL_NUM):
            for k in range(len(Xtrain[0])):
                # if prob!=0 implement function
                if DAD_std_dev[j,k]!=0:
                    val_left=1/((2*pi*DAD_std_dev[j,k])**0.5)
                    tt=-(Xtest[i,k]-mean[j,k])**2
                    bb=tt/(2*DAD_std_dev[j,k])
                    val_right=exp(bb)
                    answer=val_left*val_right
                    probs_f[j,k]=answer
            # else ignore column
                else:
                    probs_f[j,k]=0


        LL=[0]*LABEL_NUM
        for j in range(LABEL_NUM):
            SUM=0
            for k in range(len(Xtrain[0])):
                if probs_f[j,k]!=0:
                    # take sum of log values rather than product, it will avoid underflow
                    SUM+=log(probs_f[j,k],LABEL_NUM)
            LL[j]=log(labels_prob[j],LABEL_NUM)+SUM
            #sum class label prob too
        ind=LL.index(max(LL))
        Y_pred=np.append(Y_pred,ind)

    # calculate accuracy of model
    Y_pred=Y_pred.reshape(len(Y_pred),1)
    print("Accuracy for GNB= ",metrics.accuracy_score(Ytest,Y_pred)*100)

In [ ]:

# DATASET A reading file
hf=h5py.File('part_A_train.h5','r')
X=np.array(hf['X'])
Y=np.array(hf['Y'])

[r,c]=Y.shape
yy=np.array([])
for i in range(r):
    for j in range(c):
        if Y[i,j]==1:
            yy=np.append(yy,[int(j)])


Y=yy.reshape(-1,1)
# split data 80:20
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X, Y, test_size=0.2, random_state=0,stratify=Y )
gnb_(Xtrain,Xtest,Ytrain,Ytest,10)
# call gnb function on dataset a

Accuracy for GNB=  63.92857142857142


In [ ]:

# DATASET B reading file
hf=h5py.File('part_B_train.h5','r')
X=np.array(hf['X'])
Y=np.array(hf['Y'])

[r,c]=Y.shape
yy=np.array([])
for i in range(r):
    for j in range(c):
        if Y[i,j]==1:
            yy=np.append(yy,[int(j)])


Y=yy.reshape(-1,1)
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X, Y, test_size=0.2, random_state=0 )
# call gnb function on dataset b
gnb_(Xtrain,Xtest,Ytrain,Ytest,2)

Accuracy for GNB=  56.42857142857143
